<a href="https://colab.research.google.com/github/Tulipsa-Mallick/Task3_Neural_Style_Transfer/blob/main/neural_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 Install required libraries
!pip install -q tensorflow matplotlib
import matplotlib.pyplot as plt


In [ ]:
# 📌 Import Libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
from tensorflow.keras.applications import vgg19
from tensorflow.keras.preprocessing import image as kp_image
from google.colab import files

In [ ]:
# 📌 Preprocessing Functions

def load_img(path_to_img):
    target_size = (512, 512)  # Force fixed shape
    img = Image.open(path_to_img)
    img = img.convert('RGB')
    img = img.resize(target_size, Image.LANCZOS)  # consistent resize
    img = kp_image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return tf.keras.applications.vgg19.preprocess_input(img)


def deprocess_img(processed_img):
    x = processed_img.copy()
    if len(x.shape) == 4:
        x = tf.squeeze(x, 0)  # Remove batch dimension
    x = x.numpy()  # Tensor to NumPy
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]  # BGR to RGB
    x = np.clip(x, 0, 255).astype('uint8')
    return x




In [ ]:
    from google.colab import files
# 📌 Upload content and style images
uploaded = files.upload()
content_path = list(uploaded.keys())[0]
uploaded = files.upload()
style_path = list(uploaded.keys())[0]

# Load both using your defined function
content_image = load_img(content_path)
style_image = load_img(style_path)





In [ ]:
content_image = load_img(content_path)
style_image = load_img(style_path)


In [ ]:
# 📌 Display uploaded images
def imshow(img, title=None):
    out = deprocess_img(img)
    plt.imshow(out)
    if title:
        plt.title(title)
    plt.axis('off')
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
imshow(content_image, 'Content Image')
plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image')
plt.show()


In [ ]:
# 📌 Build Model to Extract Style and Content
vgg = vgg19.VGG19(include_top=False, weights='imagenet')
vgg.trainable = False

In [ ]:
# Layer configs
style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
content_layers = ['block5_conv2']
num_style_layers = len(style_layers)
num_content_layers = len(content_layers)

def get_model():
    outputs = [vgg.get_layer(name).output for name in (style_layers + content_layers)]
    model = tf.keras.Model([vgg.input], outputs)
    return model

def gram_matrix(input_tensor):
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)

In [ ]:
# 📌 Style and Content Features
def get_features(model, content_img, style_img):
    content_outputs = model(content_img)
    style_outputs = model(style_img)

    style_features = [gram_matrix(style_layer) for style_layer in style_outputs[:num_style_layers]]
    content_features = content_outputs[num_style_layers:]
    return style_features, content_features

In [ ]:
# 📌 Compute Loss
def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
    input_tensor = tf.concat([init_image], axis=0)
    outputs = model(input_tensor)

    style_output_features = outputs[:num_style_layers]
    content_output_features = outputs[num_style_layers:]

    style_score = 0
    content_score = 0

    weight_per_style_layer = 1.0 / num_style_layers
    for target_style, comb_style in zip(gram_style_features, style_output_features):
        style_score += weight_per_style_layer * tf.reduce_mean(tf.square(gram_matrix(comb_style) - target_style))

    weight_per_content_layer = 1.0 / num_content_layers
    for target_content, comb_content in zip(content_features, content_output_features):
        content_score += weight_per_content_layer * tf.reduce_mean(tf.square(comb_content - target_content))

    style_weight, content_weight = loss_weights
    total_loss = style_weight * style_score + content_weight * content_score

    return total_loss, style_score, content_score


In [ ]:
# 📌 Style Transfer with Optimizer
@tf.function()
def compute_grads(cfg):
    with tf.GradientTape() as tape:
        all_loss = compute_loss(**cfg)
    total_loss = all_loss[0]
    return tape.gradient(total_loss, cfg['init_image']), all_loss

In [ ]:
# 📌 Optimization Loop
def run_style_transfer(content_img, style_img,
                       num_iterations=1000,
                       content_weight=1e3,
                       style_weight=1e-2):

    model = get_model()
    for layer in model.layers:
        layer.trainable = False

    style_features, content_features = get_features(model, content_img, style_img)
    init_image = tf.Variable(content_img, dtype=tf.float32)

    opt = tf.optimizers.Adam(learning_rate=5.0)

    best_loss, best_img = float('inf'), None

    loss_weights = (style_weight, content_weight)
    cfg = {
        'model': model,
        'loss_weights': loss_weights,
        'init_image': init_image,
        'gram_style_features': style_features,
        'content_features': content_features
    }

    for i in range(num_iterations):
        grads, all_loss = compute_grads(cfg)
        total_loss, style_score, content_score = all_loss
        opt.apply_gradients([(grads, init_image)])
        clipped = tf.clip_by_value(init_image, -103.939, 255.0 - 103.939)
        init_image.assign(clipped)

        if total_loss < best_loss:
            best_loss = total_loss
            best_img = init_image.numpy()

        if i % 100 == 0:
            print(f"Iteration {i}:")
            print(f"  Total loss: {total_loss:.4e}")
            print(f"  Style loss: {style_score:.4e}")
            print(f"  Content loss: {content_score:.4e}")

    return best_img, best_loss



In [ ]:
# 📌 Run Style Transfer
final_img, final_loss = run_style_transfer(content_image, style_image,num_iterations=800)
print("Style transfer complete.")


In [ ]:
# 📌 Show and Save Final Output
plt.figure(figsize=(10,10))
imshow(final_img, "Final Output Image")
plt.show()
